# Pandas и большие файлы

In [1]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [2]:
df = pd.read_csv('sales_db.csv')
df

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3
5,2,Авто,35
6,3,Дети,8
7,9,Дети,17


In [13]:
d = [['1', 'Электроника', '87'], ['1', 'Дача', '10'], ['1', 'Дети', '2'], ['1', 'Дача', '199']]
max([int(x[2]) for x in d])

199

In [20]:
[i[1] for i in d if int(i[2]) == max([int(x[2]) for x in d])]

['Дача']

In [14]:
def calc(user_data):
    return max([int(x[2]) for x in user_data])

In [21]:
def calc_2(user_data):
    return [i[1] for i in d if int(i[2]) == max([int(x[2]) for x in user_data])]

In [23]:
previous_user = None
user_data = []

with open('sales_db.csv') as f:
    f.readline()
    for line in f:
        user_id, category, cost = line.strip().split(',')
        # print(user_id, category, cost)
        
        if previous_user:
            if user_id == previous_user:
                user_data.append([user_id, category, cost])
            else:
                # print(user_data)
                print(f'Пользователь {previous_user}, покупка {calc(user_data)}, категория {calc_2(user_data)}')
                user_data = [[user_id, category, cost]]
                previous_user = user_id
        else:
            previous_user = user_id
            user_data.append([user_id, category, cost])
            
print(f'Пользователь {previous_user}, покупка {calc(user_data)}, категория {calc_2(user_data)}')

Пользователь 1, покупка 199, категория ['Дача']
Пользователь 2, покупка 35, категория []
Пользователь 3, покупка 8, категория []
Пользователь 9, покупка 17, категория []


In [4]:
df.groupby('user_id').agg({'cost': 'max'})

,cost
user_id,
1,199
2,35
3,8
9,17


# Объединение датафреймов

### Данные со слайдов

In [ ]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

In [ ]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

In [ ]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

In [ ]:
visits.groupby('user_id').count().reset_index()

In [ ]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

In [ ]:
purchases_pivot.reset_index()

In [ ]:
visits_grouped.join(purchases_pivot)

### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [ ]:
visits_grouped.join(purchases_pivot, how='left')

### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [ ]:
visits_grouped.join(purchases_pivot, how='right')

### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [24]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [25]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [26]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [27]:
df_clients_earnings = pd.merge(clients, earnings, on='id', how='left') 
df_clients_earnings_spending = pd.merge(df_clients_earnings, spending, on='id', how='left') 
df_clients_earnings_spending.loc[df_clients_earnings_spending.debit<df_clients_earnings_spending.credit]


,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


In [28]:
joined = (
    clients
    .merge(earnings, on='id', how='left')
    .merge(spending, on='id', how='left')
)

In [30]:
joined[joined.debit < joined.credit]

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [ ]:
visits_grouped.join(purchases_pivot, how='inner')

### Outer join
Оставляет все строчки

In [ ]:
visits_grouped.join(purchases_pivot, how='outer')

# Конкатенация таблиц

In [31]:
'123' + '456'

'123456'

In [32]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [33]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [34]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [37]:
pd.concat([a, b]).reset_index()

/Users/kbashevoy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,index,date,value_a,value_b
0,0,2020-01-01,1.0,NaN
1,1,2020-01-02,2.0,NaN
2,2,2020-01-03,3.0,NaN
3,0,2020-01-01,NaN,3.0
4,1,2020-01-02,NaN,4.0
5,2,2020-01-03,NaN,5.0


In [38]:
# объединение по горизонтали
pd.concat([a, b], axis=1)['date']

,date,date
0,2020-01-01,2020-01-01
1,2020-01-02,2020-01-02
2,2020-01-03,2020-01-03


### Дубликаты при объединении таблиц

In [44]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [45]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [46]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [47]:
ratings.merge(movies, how='inner', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [48]:
movies.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [49]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [ ]:
ratings.merge(movies, how = 'right', on = 'movieId')

### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [56]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [59]:
visits.merge(orders)

,date,source,visits,orders
0,2019-11-01,organic,16825,198
1,2019-11-01,paid,1952,198
2,2019-11-02,organic,21890,225
3,2019-11-02,paid,376,225
4,2019-11-03,organic,19509,201


In [51]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [58]:
visits.groupby('date').visits.sum().reset_index()

,date,visits
0,2019-11-01,18777
1,2019-11-02,22266
2,2019-11-03,19509


In [52]:
visits.groupby('date').visits.sum().reset_index().merge(orders, on='date', how='left')

,date,visits,orders
0,2019-11-01,18777,198
1,2019-11-02,22266,225
2,2019-11-03,19509,201


In [53]:
visits.drop_duplicates(subset = 'date', keep = 'first', inplace = True) 
visits

,date,source,visits
0,2019-11-01,organic,16825
2,2019-11-02,organic,21890
4,2019-11-03,organic,19509


In [54]:
visits.merge(orders, how = 'left', on = 'date')

,date,source,visits,orders
0,2019-11-01,organic,16825,198
1,2019-11-02,organic,21890,225
2,2019-11-03,organic,19509,201


In [62]:
visit_pivot = visits.pivot_table(index='date', columns='source', values='visits', aggfunc='sum').reset_index()
visit_pivot

source,date,organic,paid
0,2019-11-01,16825.0,1952.0
1,2019-11-02,21890.0,376.0
2,2019-11-03,19509.0,NaN


In [65]:
visit_pivot['visits'] = visit_pivot['organic'] + visit_pivot['paid']

In [66]:
visit_pivot.merge(orders)

,date,organic,paid,visits,orders
0,2019-11-01,16825.0,1952.0,18777.0,198
1,2019-11-02,21890.0,376.0,22266.0,225
2,2019-11-03,19509.0,NaN,NaN,201


### Оптимизация хранения данных

In [67]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [68]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [70]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [73]:
a = 1029

In [74]:
a.__sizeof__()

28

In [72]:
len(ratings)

100004

In [76]:
s = 'Animation|Children|Drama|Musical'

In [77]:
s.__sizeof__()

81

In [75]:
movies[movies.movieId == 1029]

,movieId,title,genres
833,1029,Dumbo (1941),Animation|Children|Drama|Musical


In [71]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [78]:
len(movies)

9125

In [ ]:
len(ratings) == len(joined)

In [69]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [ ]:
import numpy as np

In [ ]:
genres = ['Drama', 'Action', 'Thriller']

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

In [ ]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

### Считаем рейтинг жанров

In [ ]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

In [ ]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [ ]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base